In [8]:
!pip install nbimporter

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import nbimporter
from dataset import load_datasets 
from model import get_model  

In [7]:
import torch
import torch.optim as optim
import torch
import torch.nn as nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_loader, val_loader, test_loader = load_datasets("pedestrian_risk_analysis.csv")
model = get_model(device)

In [ ]:
import torch
import torch.optim as optim
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
_loss_plot_test=[]

num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    total_loss, correct = 0, 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        correct += (outputs.argmax(1) == labels).sum().item()
    
    avg_loss = total_loss/ len(train_loader.dataset)
    train_accuracy = correct / len(train_loader.dataset)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss:.4f}, Accuracy: {train_accuracy:.4f}")
    with torch.no_grad():
        prediction, truth = zip(*[(model(x.to(device)), y.to(device)) for x, y in val_loader])
        prediction = torch.cat(prediction, dim=0)
        truth = torch.cat(truth, dim=0)
        val_loss = loss_fn(prediction, truth)
        _loss_plot_test.append(val_loss.item())
        
    print(f"Epoch [{epoch+1}/{num_epochs}] - Loss: {avg_loss:.4f} Val Loss: {val_loss:.4f}")

Epoch [1/10], Loss: 188.6011, Accuracy: 0.6856
Epoch [1/10] - Loss: 0.0219 Val Loss: 0.5502
Epoch [2/10], Loss: 115.2547, Accuracy: 0.8235
Epoch [2/10] - Loss: 0.0134 Val Loss: 0.4243
Epoch [3/10], Loss: 89.6474, Accuracy: 0.8681
Epoch [3/10] - Loss: 0.0104 Val Loss: 0.3496
Epoch [4/10], Loss: 73.3986, Accuracy: 0.8907
Epoch [4/10] - Loss: 0.0085 Val Loss: 0.3107
Epoch [5/10], Loss: 63.2544, Accuracy: 0.9034
Epoch [5/10] - Loss: 0.0073 Val Loss: 0.2736
Epoch [6/10], Loss: 54.8403, Accuracy: 0.9136
Epoch [6/10] - Loss: 0.0064 Val Loss: 0.2651
Epoch [7/10], Loss: 49.8203, Accuracy: 0.9270
Epoch [7/10] - Loss: 0.0058 Val Loss: 0.2420
Epoch [8/10], Loss: 45.6762, Accuracy: 0.9329
Epoch [8/10] - Loss: 0.0053 Val Loss: 0.2983
Epoch [9/10], Loss: 41.9194, Accuracy: 0.9360
